In [1]:
import openai
import pandas as pd
import numpy as np

In [2]:
openai.api_key = "sk-X2HFbojkXplIAcfYQ5X946Dq62vbAEug5s5yvshh"

### Create prompt

In [3]:
intro = "Below we classify tweets by their sentiment as either being Positive, Negative or Neutral\n"
ex1 = "Tweet: @VirginAmerica it's really aggressive to blast obnoxious 'entertainment' in your guests' faces &amp; they have little recourse\
\nSentiment: Negative\n"
ex2 = "Tweet: @united Just submitted my response on the link you sent.\
\nSentiment: Neutral\n"
ex3 = "Tweet: @VirginAmerica I love the hipster innovation. You are a feel good brand.\
\nSentiment: Positive\n"

#### [dummy call to the api]

In [4]:
test = "@VirginAmerica this is great news! \
America could start flights to Hawaii by end of year http://t.co/r8p2Zy3fe4 via @Pacificbiznews"

In [5]:
prompt = intro + ex1 + ex2 + ex3 + "Tweet: " + test + "\nSentiment:"
response = openai.Completion.create(engine="davinci",
                                    prompt=prompt,
                                    top_p=0.1,
                                    logprobs=3,
                                    max_tokens=1)
response['choices'][0]['text']

' Positive'

### Grab data and sample a proportion randomly

In [12]:
sample_fraction = 0.01
dataset_file = "data/0.75_min_confidence.csv"

train = pd.read_csv(dataset_file)
train = train.sample(frac=sample_fraction).reset_index(drop=True)
train

,airline_sentiment,text
0,negative,@USAirways : Premier provider of missed connec...
1,positive,@SouthwestAir is making dreams come true with ...
2,negative,@USAirways they have all of this at hotels. St...
3,negative,@SouthwestAir had to Cancelled Flight my fligh...
4,negative,@AmericanAir Right. But more than two hours La...
...,...,...
100,negative,@united wont transfer flight ticket to accompa...
101,negative,@SouthwestAir went to purchase a flight that I...
102,negative,@JetBlue says the person who hasn't been delay...
103,negative,@usairways Why are we waiting 40 min so far fo...


### Run inference using GPT-3 API

In [13]:
def model(tweet):
    """ Returns gtp-3 logprobs for top 3 predictions given a tweet """
    prompt = intro + ex1 + ex2 + "Tweet: " + tweet + "\nSentiment:"
    response = openai.Completion.create(engine="davinci",
                                        prompt=prompt,
                                        temperature=0.05,
                                        logprobs=3,
                                        max_tokens=1)
    return pd.Series(
        dict(response['choices'][0]['logprobs']['top_logprobs'][0]))

In [14]:
def sigmoid(x):
    """ Converts logprobs into a more human readable probability """
    return 2 / (1 + np.exp(-x))

In [15]:
def evaluate(x):
    """ Returns the highest scoring prediction """
    return x.idxmax().lower()[1:]

In [16]:
probs = train['text'].apply(model).applymap(sigmoid)
probs['prediction'] = probs.apply(evaluate, axis=1)
result = train.merge(probs, left_index=True, right_index=True)

### Calculate the accuracy

In [17]:
loss = result['airline_sentiment'] == result['prediction']
loss.agg('sum') / loss.size

0.8285714285714286